In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Nodule"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path=None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
405: 4375
80: 4375
645: 4375
1057: 4375
605: 4375
4375: 4375
41: 4375
232: 4375
289: 4375
92: 4375
71: 4375
114: 4375
274: 4375
6: 4375
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
4354/4375 [============================>.] - ETA: 0s - loss: 4.8611 - leftLayer1_loss: 0.0775 - midLayer1_loss: 1.4074 - rightLayer1_loss: 0.8036 - leftLayer2_loss: 0.0490 - midLayer2_loss: 1.7920 - rightLayer2_loss: 0.7315
Epoch 00006: val_loss improved from 4.85803 to 4.80794, saving model to ./experiments/0.009_weights.h5
4375/4375 [==============================] - 11s 2ms/step - loss: 4.8602 - leftLayer1_loss: 0.0775 - midLayer1_loss: 1.4073 - rightLayer1_loss: 0.8035 - leftLayer2_loss: 0.0489 - midLayer2_loss: 1.7917 - rightLayer2_loss: 0.7313 - val_loss: 4.8079 - val_leftLayer1_loss: 0.0756 - val_midLayer1_loss: 1.4052 - val_rightLayer1_loss: 0.8287 - val_leftLayer2_loss: 0.0665 - val_midLayer2_loss: 1.5638 - val_rightLayer2_loss: 0.8681
Epoch 7/11
4371/4375 [============================>.] - ETA: 0s - loss: 4.8267 - leftLayer1_loss: 0.0723 - midLayer1_loss: 1.4075 - rightLayer1_loss: 0.7833 - leftLayer2_loss: 0.0450 - midLayer2_loss: 1.7947 - rightLayer2_loss: 0.7240

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 4375 steps, validate for 613 steps
Epoch 1/11
4373/4375 [============================>.] - ETA: 0s - loss: 5.9636 - leftLayer1_loss: 0.1186 - midLayer1_loss: 1.3888 - rightLayer1_loss: 1.5400 - leftLayer2_loss: 0.1194 - midLayer2_loss: 1.5092 - rightLayer2_loss: 1.2878
Epoch 00001: val_loss improved from inf to 5.49640, saving model to ./experiments/0.009999999999999998_weights.h5
4375/4375 [==============================] - 12s 3ms/step - loss: 5.9634 - leftLayer1_loss: 0.1186 - midLayer1_loss: 1.3888 - rightLayer1_loss: 1.5398 - leftLayer2_loss: 0.1194 - midLayer2_loss: 1.5092 - rightLayer2_loss: 1.2876 - val_loss: 5.4964 - val_leftLayer1_loss: 0.1120 - val_midLayer1_loss: 1.3955 - val_rightLayer1_loss: 1.2711 - val_leftLayer2_loss: 0.1117 - val_midLayer2_loss: 1.3984 - val_rightLayer2_loss: 1.

4370/4375 [============================>.] - ETA: 0s - loss: 4.4322 - leftLayer1_loss: 0.0537 - midLayer1_loss: 1.3890 - rightLayer1_loss: 0.7306 - leftLayer2_loss: 0.0381 - midLayer2_loss: 1.5092 - rightLayer2_loss: 0.7117
Epoch 00011: val_loss improved from 4.52046 to 4.50513, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Nodule: 0.5829634702758626

auprc 0Nodule: 0.08316849298769124

auroc 1Nodule: 0.5276756623184902

auprc 1Nodule: 0.06279376877684072

auroc 2Nodule: 0.5230102219604393

auprc 2Nodule: 0.05901149327250921

auroc 3Nodule: 0.5908832439874842

auprc 3Nodule: 0.08424306869058538

auroc 4Nodule: 0.3521402706754958

auprc 4Nodule: 0.04230653138387535

auroc 5Nodule: 0.6000339418366155

auprc 5Nodule: 0.07748076920036157

mean auroc: 0.5294511351757313

mean auprc: 0.06816735405197726

max auroc: 0.6000339418366155

max auprc: 0.

Epoch 4/11
4366/4375 [============================>.] - ETA: 0s - loss: 4.6373 - leftLayer1_loss: 0.0891 - midLayer1_loss: 1.3330 - rightLayer1_loss: 0.8636 - leftLayer2_loss: 0.0623 - midLayer2_loss: 1.5294 - rightLayer2_loss: 0.7600
Epoch 00004: val_loss improved from 4.83703 to 4.72860, saving model to ./experiments/0.010999999999999998_weights.h5
4375/4375 [==============================] - 11s 3ms/step - loss: 4.6375 - leftLayer1_loss: 0.0891 - midLayer1_loss: 1.3330 - rightLayer1_loss: 0.8636 - leftLayer2_loss: 0.0623 - midLayer2_loss: 1.5295 - rightLayer2_loss: 0.7600 - val_loss: 4.7286 - val_leftLayer1_loss: 0.0854 - val_midLayer1_loss: 1.3332 - val_rightLayer1_loss: 0.8694 - val_leftLayer2_loss: 0.0768 - val_midLayer2_loss: 1.4336 - val_rightLayer2_loss: 0.9303
Epoch 5/11
4363/4375 [============================>.] - ETA: 0s - loss: 4.5542 - leftLayer1_loss: 0.0815 - midLayer1_loss: 1.3315 - rightLayer1_loss: 0.8202 - leftLayer2_loss: 0.0549 - midLayer2_loss: 1.5219 - rightLaye

...

4370/4375 [============================>.] - ETA: 0s - loss: 4.2940 - leftLayer1_loss: 0.0568 - midLayer1_loss: 1.3561 - rightLayer1_loss: 0.7440 - leftLayer2_loss: 0.0374 - midLayer2_loss: 1.3881 - rightLayer2_loss: 0.7115
Epoch 00011: val_loss improved from 4.41176 to 4.39590, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Nodule: 0.7027865324756983

auprc 0Nodule: 0.17196485647360554

auroc 1Nodule: 0.37919834068443925

auprc 1Nodule: 0.04497395040190226

auroc 2Nodule: 0.6306193710606078

auprc 2Nodule: 0.09918901867793048

auroc 3Nodule: 0.5819622926077449

auprc 3Nodule: 0.07461902944841772

auroc 4Nodule: 0.4120017943728269

auprc 4Nodule: 0.046216487581625935

auroc 5Nodule: 0.5354623847406592

auprc 5Nodule: 0.06291501468243527

mean auroc: 0.540338452656996

mean auprc: 0.08331305954431954

max auroc: 0.7027865324756983

max auprc: 0

Epoch 4/11
4353/4375 [============================>.] - ETA: 0s - loss: 4.5914 - leftLayer1_loss: 0.0988 - midLayer1_loss: 1.3431 - rightLayer1_loss: 0.9344 - leftLayer2_loss: 0.0633 - midLayer2_loss: 1.3900 - rightLayer2_loss: 0.7617
Epoch 00004: val_loss improved from 4.82068 to 4.69161, saving model to ./experiments/0.012999999999999996_weights.h5
4375/4375 [==============================] - 11s 2ms/step - loss: 4.5910 - leftLayer1_loss: 0.0988 - midLayer1_loss: 1.3430 - rightLayer1_loss: 0.9341 - leftLayer2_loss: 0.0633 - midLayer2_loss: 1.3902 - rightLayer2_loss: 0.7615 - val_loss: 4.6916 - val_leftLayer1_loss: 0.0956 - val_midLayer1_loss: 1.3478 - val_rightLayer1_loss: 0.9250 - val_leftLayer2_loss: 0.0803 - val_midLayer2_loss: 1.3164 - val_rightLayer2_loss: 0.9265
Epoch 5/11
4357/4375 [============================>.] - ETA: 0s - loss: 4.5014 - leftLayer1_loss: 0.0920 - midLayer1_loss: 1.3433 - rightLayer1_loss: 0.8742 - leftLayer2_loss: 0.0558 - midLayer2_loss: 1.3905 - rightLaye

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 4375 steps, validate for 613 steps
Epoch 1/11
4353/4375 [============================>.] - ETA: 0s - loss: 5.9279 - leftLayer1_loss: 0.1189 - midLayer1_loss: 1.3506 - rightLayer1_loss: 1.5585 - leftLayer2_loss: 0.1117 - midLayer2_loss: 1.4808 - rightLayer2_loss: 1.3072
Epoch 00001: val_loss improved from inf to 5.40873, saving model to ./experiments/0.013999999999999995_weights.h5
4375/4375 [==============================] - 11s 3ms/step - loss: 5.9242 - leftLayer1_loss: 0.1189 - midLayer1_loss: 1.3506 - rightLayer1_loss: 1.5571 - leftLayer2_loss: 0.1116 - midLayer2_loss: 1.4807 - rightLayer2_loss: 1.3053 - val_loss: 5.4087 - val_leftLayer1_loss: 0.1129 - val_midLayer1_loss: 1.3523 - val_rightLayer1_loss: 1.2913 - val_leftLayer2_loss: 0.1066 - val_midLayer2_loss: 1.3350 - val_rightLayer2_loss: 1.

4367/4375 [============================>.] - ETA: 0s - loss: 4.3671 - leftLayer1_loss: 0.0541 - midLayer1_loss: 1.3496 - rightLayer1_loss: 0.7344 - leftLayer2_loss: 0.0376 - midLayer2_loss: 1.4788 - rightLayer2_loss: 0.7126
Epoch 00011: val_loss improved from 4.41820 to 4.40278, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Nodule: 0.5754144649740484

auprc 0Nodule: 0.07674711570011476

auroc 1Nodule: 0.42335755772153705

auprc 1Nodule: 0.04695281318536163

auroc 2Nodule: 0.5666084755890382

auprc 2Nodule: 0.07405042504545142

auroc 3Nodule: 0.5086750150803296

auprc 3Nodule: 0.05895444323078333

auroc 4Nodule: 0.27367235405454055

auprc 4Nodule: 0.0371705263956547

auroc 5Nodule: 0.49887576542214446

auprc 5Nodule: 0.05464561575478753

mean auroc: 0.4744339388069397

mean auprc: 0.058086823218692234

max auroc: 0.5754144649740484

max auprc:

Epoch 4/11
4362/4375 [============================>.] - ETA: 0s - loss: 4.5102 - leftLayer1_loss: 0.0967 - midLayer1_loss: 1.3119 - rightLayer1_loss: 0.9037 - leftLayer2_loss: 0.0629 - midLayer2_loss: 1.3760 - rightLayer2_loss: 0.7590
Epoch 00004: val_loss improved from 4.72838 to 4.60829, saving model to ./experiments/0.014999999999999994_weights.h5
4375/4375 [==============================] - 11s 2ms/step - loss: 4.5097 - leftLayer1_loss: 0.0967 - midLayer1_loss: 1.3119 - rightLayer1_loss: 0.9036 - leftLayer2_loss: 0.0628 - midLayer2_loss: 1.3758 - rightLayer2_loss: 0.7588 - val_loss: 4.6083 - val_leftLayer1_loss: 0.0932 - val_midLayer1_loss: 1.3156 - val_rightLayer1_loss: 0.9026 - val_leftLayer2_loss: 0.0776 - val_midLayer2_loss: 1.2990 - val_rightLayer2_loss: 0.9204
Epoch 5/11
4362/4375 [============================>.] - ETA: 0s - loss: 4.4255 - leftLayer1_loss: 0.0895 - midLayer1_loss: 1.3126 - rightLayer1_loss: 0.8510 - leftLayer2_loss: 0.0553 - midLayer2_loss: 1.3738 - rightLaye

In [17]:
print(np.max(maxi))

0.7027865324756983
